In [2]:
from sqlalchemy import create_engine, text
import json
import os

user = os.environ['POSTGRES_USER']
password = os.environ['POSTGRES_PASSWORD']
db   = os.environ['POSTGRES_DB']
host   = "localhost"

In [3]:

engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}/{db}",  future=True)

In [25]:
with engine.connect() as conn:
    stmt = """CREATE TABLE records_temp (id serial PRIMARY KEY, 
                                    time TIMESTAMP, clientMac VARCHAR,
                                    ipv4 VARCHAR, ssid VARCHAR,
                                    os VARCHAR, manufacturer VARCHAR,
                                    nearestApMac VARCHAR, nearestApRssi SMALLINT)"""
    conn.execute(text(stmt))
    conn.commit()

2022-02-02 13:23:11,265 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-02 13:23:11,268 INFO sqlalchemy.engine.Engine CREATE TABLE records_temp (id serial PRIMARY KEY, 
                                    time TIMESTAMP, clientMac VARCHAR,
                                    ipv4 VARCHAR, ssid VARCHAR,
                                    os VARCHAR, manufacturer VARCHAR,
                                    nearestApMac VARCHAR, nearestApRssi SMALLINT)
2022-02-02 13:23:11,268 INFO sqlalchemy.engine.Engine [generated in 0.00350s] {}
2022-02-02 13:23:11,282 INFO sqlalchemy.engine.Engine COMMIT


In [28]:
with engine.connect() as conn:
    conn.execute(
        text("INSERT INTO records_temp (time, clientMac,nearestApRssi) VALUES (:time, :clientMac, :nearestApRssi)"),
        [{"time": "2022-01-25 06:11:22", "clientMac": "7c:2e:bd:0b:b2:bf","nearestApRssi":-51}, {"time": "2022-01-25 06:11:38", "clientMac": "38:8b:59:48:21:ad","nearestApRssi":-51}]
    )
    conn.commit()

2022-02-02 13:25:32,068 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-02 13:25:32,072 INFO sqlalchemy.engine.Engine INSERT INTO records_temp (time, clientMac,nearestApRssi) VALUES (%(time)s, %(clientMac)s, %(nearestApRssi)s)
2022-02-02 13:25:32,075 INFO sqlalchemy.engine.Engine [cached since 38.47s ago] ({'time': '2022-01-25 06:11:22', 'clientMac': '7c:2e:bd:0b:b2:bf', 'nearestApRssi': -51}, {'time': '2022-01-25 06:11:38', 'clientMac': '38:8b:59:48:21:ad', 'nearestApRssi': -51})
2022-02-02 13:25:32,086 INFO sqlalchemy.engine.Engine COMMIT


In [29]:
with open('../scripts/postExampleV3.json') as f:
    data = json.load(f)
    with engine.connect() as conn:
        for row in data['data']['observations']:
            for key,value in row['latestRecord'].items():
                row[key] = value
            stmt = """INSERT INTO records_temp (time, clientmac, ipv4, ssid, os, manufacturer, nearestApMac,nearestApRssi)
                    VALUES (:time, :clientmac, :ipv4, :ssid, :os, :manufacturer, :nearestApMac, :nearestApRssi)"""
            conn.execute(text(stmt),[{"time":" ".join(row["time"][:-1].split('T')), "clientmac":row["clientMac"],"ipv4":row["ipv4"],
                                    "ssid":row["ssid"],"os":row['os'],"manufacturer":row["manufacturer"],
                                    "nearestApMac":row['nearestApMac'],"nearestApRssi": int(row['nearestApRssi'])}]) 
        conn.commit()


2022-02-02 13:26:44,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-02 13:26:44,526 INFO sqlalchemy.engine.Engine INSERT INTO records_temp (time, clientmac, ipv4, ssid, os, manufacturer, nearestApMac,nearestApRssi)
                    VALUES (%(time)s, %(clientmac)s, %(ipv4)s, %(ssid)s, %(os)s, %(manufacturer)s, %(nearestApMac)s, %(nearestApRssi)s)
2022-02-02 13:26:44,531 INFO sqlalchemy.engine.Engine [cached since 454.5s ago] {'time': '2022-01-25 06:11:22', 'clientmac': '7c:2e:bd:0b:b2:bf', 'ipv4': None, 'ssid': None, 'os': None, 'manufacturer': 'Google', 'nearestApMac': 'e0:cb:bc:96:23:c0', 'nearestApRssi': -93}
2022-02-02 13:26:44,538 INFO sqlalchemy.engine.Engine INSERT INTO records_temp (time, clientmac, ipv4, ssid, os, manufacturer, nearestApMac,nearestApRssi)
                    VALUES (%(time)s, %(clientmac)s, %(ipv4)s, %(ssid)s, %(os)s, %(manufacturer)s, %(nearestApMac)s, %(nearestApRssi)s)
2022-02-02 13:26:44,539 INFO sqlalchemy.engine.Engine [cached since 454.5s a

In [16]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM records_temp"))
    for row in result:
        print(row)
        

2022-02-02 13:04:40,479 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-02 13:04:40,482 INFO sqlalchemy.engine.Engine SELECT * FROM records_temp
2022-02-02 13:04:40,484 INFO sqlalchemy.engine.Engine [cached since 130s ago] {}
(1, datetime.datetime(2022, 1, 25, 6, 11, 22), '7c:2e:bd:0b:b2:bf', None, None, None, None, None, None)
(2, datetime.datetime(2022, 1, 25, 6, 11, 38), '38:8b:59:48:21:ad', None, None, None, None, None, None)
2022-02-02 13:04:40,489 INFO sqlalchemy.engine.Engine ROLLBACK


In [15]:
with open('postExampleV3.json') as f:
    data = json.load(f)
    for row in data['data']['observations'][:3]:
        for key,value in row['latestRecord'].items():
            row[key] = value
    print(data['data']['observations'][1])
        

FileNotFoundError: [Errno 2] No such file or directory: 'postExampleV3.json'

In [5]:
with engine.connect() as conn:
    result = conn.execute(text("DROP TABLE records"))
    conn.commit()
     